In [0]:
%tensorflow_version 1.x

In [0]:
import gym
import numpy as np
import random as pr
from gym.envs.registration import register
import tensorflow as tf
tf.enable_eager_execution()

In [0]:
#강화 학습 환경을 설정
register(
    id="FrozenLake-v1",  #강화 학습 환경의 아이디를 설정
    #gym.envs.toy_text:FrozenLakeEnv : FrozenLake 환경을 설정해서 리턴 하도록 설정
    entry_point="gym.envs.toy_text:FrozenLakeEnv", 
    kwargs={
            "map_name": "4x4",  #FrozenLake 의 줄 칸을 설정 4줄 X4 칸
            "is_slippery":False #미끄러짐 효과 설정 안함
            }
)

In [0]:
#gym.make("FrozenLake-v1"): FrozenLake-v1 환경을 설정해서 env에 리턴
env=gym.make("FrozenLake-v1")

In [0]:
w=tf.Variable(tf.random_uniform([16,4], 0, 0.1))
w

In [0]:
dis=0.99
successList=[]

In [0]:
optimizer=tf.train.AdamOptimizer(0.001)

In [0]:
np.identity(16)[2:3]

In [0]:
def one_hot(num):
    arr=np.identity(16)[num:num+1]
    return np.array(arr, dtype="float32")

In [0]:
one_hot(5)

In [0]:
for step in range(2000):
    print("="*100)
    print("step:",step)
    position=env.reset()
    e=1.0/((step//100)+1)

    while True:
        with tf.GradientTape() as tape:
            position_onehot=one_hot(position)
            print("position:",position)
            predQ=tf.matmul(position_onehot,w)
            print("predQ:",predQ)
            if np.random.rand(1) <e:
                action=env.action_space.sample()
            else:
                action=np.argmax(predQ)
            
            print("action:",action)
            new_position,reward,done,info=env.step(action)

            if done:
                Qs=predQ.numpy()
                Qs[0,action]=reward
                Qs=tf.convert_to_tensor(Qs, np.float32)
                successList.append([step,reward])
                print("="*100)
                print("step:",step,":cost:",cost.numpy())
            else:
                new_position_onehot=one_hot(new_position)
                nextQ=tf.matmul(new_position_onehot,w)
                Qs=predQ.numpy()
                Qs[0,action]=reward+dis*np.max(nextQ)
                Qs=tf.convert_to_tensor(Qs,np.float32)
            
            print("Qs:",Qs)
            cost=tf.reduce_sum(tf.square(Qs-predQ))
            grads=tape.gradient(cost,[w])
            optimizer.apply_gradients(\
                            grads_and_vars=zip(grads,[w]))
            position=new_position
            print("="*100)

            if done==True:
                break
